# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_csv = '../output_data/weather_csv.csv'
weather_dataframe = pd.read_csv(weather_csv)
weather_dataframe.head()

,City,Lon,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hermanus,19.2345,-34.4187,55.80,76,95,24.63,ZA,2021-06-26
1,tuktoyaktuk,-133.0374,69.4541,44.60,87,90,18.41,CA,2021-06-26
2,saldanha,17.9442,-33.0117,59.16,78,100,14.32,ZA,2021-06-26
3,luwuk,122.7875,-0.9516,83.88,69,86,3.49,ID,2021-06-26
4,piacabucu,-36.4344,-10.4056,75.06,67,100,8.88,BR,2021-06-26


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = weather_dataframe[['Lat', 'Lon']].astype(float)
humidity = weather_dataframe['Humidity'].astype(float)

In [4]:
locations = weather_dataframe[['Lat', 'Lon']].astype(float)
weights = weather_dataframe['Humidity'].astype(float)
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create my ideal dataframe.
filtered_dataframe = weather_dataframe.loc[(weather_dataframe['Max Temp'] <= 90) & (weather_dataframe['Max Temp'] >= 60) & \
                        (weather_dataframe['Cloudiness'] <= 50) & (weather_dataframe['Humidity'] <= 40)].dropna()
filtered_dataframe.head()

,City,Lon,Lat,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,riyadh,46.7219,24.6877,82.54,11,5,1.77,SA,2021-06-26
63,mastung,66.8450,29.7994,68.97,9,0,3.02,PK,2021-06-26
89,parral,-105.6667,26.9333,89.06,18,7,14.79,MX,2021-06-26
104,atar,-13.0499,20.5169,87.44,40,21,11.36,MR,2021-06-26
109,birjand,59.2211,32.8663,66.47,38,0,9.86,IR,2021-06-26


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Make a new hotel dataframe using coordinates
hotel_dataframe = filtered_dataframe.loc[:,['City','Country','Lat','Lon']]
# Create new column for hotel name
hotel_dataframe['Hotel'] = ''
hotel_dataframe.head()

,City,Country,Lat,Lon,Hotel
15,riyadh,SA,24.6877,46.7219,
63,mastung,PK,29.7994,66.8450,
89,parral,MX,26.9333,-105.6667,
104,atar,MR,20.5169,-13.0499,
109,birjand,IR,32.8663,59.2211,


In [14]:
# Setup each url
google_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

# Create params dictionary for each iteration
params = {
    'radius': 5000,
    'types': 'lodging',
    'key': g_key
}

# Get that loop rollin!
for index, row in hotel_dataframe.iterrows():
    # Identify columns
    lat = row['Lat']
    lng = row['Lon']
    city = row['City']
    # change location each iteration while leaving original params in place
    params['location'] = f'{lat},{lng}'
    # Make API Request
    print(f'Retrieving results for index {index}: {city}.')
    response = requests.get(google_url, params=params).json()
    # Pull the results
    results = response['results']
    # Try & Except statement for saving results or skipping invalid
    try:
        print(f'Nearest hotel in {city} is {results[0]["name"]}.')
        hotel_dataframe.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print(f'-'*3, 'Invalid Result', '-'*3)
        
    print(f'-'*22)

Retrieving results for index 15: riyadh.
Nearest hotel in riyadh is InterContinental Riyadh, an IHG Hotel.
----------------------
Retrieving results for index 63: mastung.
Nearest hotel in mastung is Al Hasnan Cold Drink Santer.
----------------------
Retrieving results for index 89: parral.
Nearest hotel in parral is HOTEL LOS ARCOS PARRAL.
----------------------
Retrieving results for index 104: atar.
Nearest hotel in atar is Oumou Elghoura.
----------------------
Retrieving results for index 109: birjand.
Nearest hotel in birjand is Birjand Tourist Hotel.
----------------------
Retrieving results for index 123: tahta.
Nearest hotel in tahta is El Salam Hotel.
----------------------
Retrieving results for index 136: cherskiy.
Nearest hotel in cherskiy is Gostinitsa.
----------------------
Retrieving results for index 144: jalu.
Nearest hotel in jalu is صاحبة الفخامة.
----------------------
Retrieving results for index 246: east london.
Nearest hotel in east london is Tu Casa.
-------

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = '''
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
'''
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_dataframe.iterrows()]
locations = hotel_dataframe[['Lat', 'Lon']]

In [16]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))